In [4]:
#Python program to scrape fresher job listings from ‘https://internshala.com/’.
#Job title, Company name, CTC, and Apply date
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup

no_pages = 2

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://internshala.com/fresher-jobs/'+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []
    
    for d in soup.findAll('div', attrs={'class':'container-fluid individual_internship visibilityTrackerItem'}):
        #print(d)
        job_title = d.find('div', attrs={'class':'heading_4_5 profile'})
        company = d.find('a', attrs={'class':'link_display_like_text'})
        ctc_list=d.find_all('div', attrs={'class':'item_body'})

        all1=[]

        if job_title is not None:
            #print(job_title.text)
            all1.append(job_title.text.replace("\n",""))
        else:
            all1.append("unknown-product")

        if company is not None:
            #print(company.text)
            all1.append(company.text.replace("\n","").replace("                        ",""))
        else:
            all1.append('not specified')
        
        ctc=[]
        for i in ctc_list:
            ctc.append(i.text.replace("\n",""))
        list=[]
        for i in range(len(ctc)):
            if i%3==0:
                continue
            else:
                list.append(ctc[i])
        ctc=[]
        apply=[]
        for i in range(len(list)):
            if i%2==0:
                ctc.append(list[i].strip())
            else:
                apply.append(list[i])
        
        if ctc is not None:
            #print(ctc)
            all1.append(ctc)
        else:
            all1.append('not specified')
            
        if apply is not None:
            #print(apply)
            all1.append(apply)
        else:
            all1.append('not specified')
        alls.append(all1)
    return alls

results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Job Title', 'Company Name','CTC','Apply By'])
df['CTC'] = df['CTC'].str[0]
df['Apply By']=df['Apply By'].str[0]
df.index +=1
df.to_csv('intern_products.csv', index=False, encoding='utf-8')
df=df.style.set_properties(**{'text-align': 'left'})
df